In [1]:
import tkinter as tk
import numpy as np
import copy

In [2]:
class neuralNetwork_1:
    
    def __init__(self,neurons_layers,l_rate):
        self.n_layers = neurons_layers
        self.lr = l_rate
        self.sigmoid = lambda x: 1/(1+np.exp(-x))
        self.W = []
        for i in range(len(self.n_layers)-1):
            self.W.append(np.random.normal(0.0, pow(self.n_layers[i+1], -0.5),\
                                           (self.n_layers[i+1], self.n_layers[i]+1)))
        self.len_W = len(self.W)
            
    def __prepare(self,inputs_list):
        inputs = np.array(inputs_list, ndmin=2).T
        inputs = np.append(inputs,np.ones((1,inputs.shape[1])),axis=0) 
        out_list = [inputs]
        return out_list
    
    def __forward_prop(self,out_list):
        for i in range(self.len_W):
            out_list.append(self.W[i]@out_list[-1])
            out_list[-1] = self.sigmoid(out_list[-1])
            out_list[-1] = np.append(out_list[-1],np.ones((1,out_list[-1].shape[1])),axis=0)
    
    def __back_prop(self,er_list,out_list):
        for i in np.arange(self.len_W-1,-1,-1):
            er_list.append(self.W[i].T@er_list[-1][:-1])
            self.W[i] += self.lr*(er_list[-2][:-1]*out_list[i+1][:-1] * \
                                  (1-out_list[i+1][:-1]))@out_list[i].T
        
    def train(self,inputs_list,targets_list):
        out_list = self.__prepare(inputs_list)
        targets = np.array(targets_list, ndmin=2).T
        targets = np.append(targets,np.ones((1,targets.shape[1])),axis=0)
        
        self.__forward_prop(out_list)
        er_list = [targets - out_list[-1]]
        self.__back_prop(er_list,out_list)

    def query(self,inputs_list):
        out_list = self.__prepare(inputs_list)
        self.__forward_prop(out_list)
        return out_list[-1][:-1]

In [3]:
size_p = (28,28)
neurons_layers = [size_p[0]*size_p[1],1000,16]
learning_rate = 0.01
ratio = size_p[1] / size_p[0]
n = neuralNetwork_1(neurons_layers,learning_rate)

In [4]:
# импорт из файла
extract_W = []
with open('+28x28_weights_9799_1000_16_0.01_2.npy', 'rb') as n_W:
    while True:
        try:
            extract_W.append(np.load(n_W))
        except:
            break
n.W = extract_W

In [5]:
class App(tk.Tk):
    def __init__(self):
        super().__init__()
        self.canv = tk.Canvas(self, width = 300, height = 300)
        self.button_clear = tk.Button(self,text='clear all',command=self.btn_clear)
        self.button_back = tk.Button(self,text='<-',command=self.btn_back)
        self.button_erase = tk.Button(self,text='erase',command=self.btn_erase)
        self.button_draw = tk.Button(self,text='draw',command=self.btn_draw)
        self.button_forward = tk.Button(self,text='->',command=self.btn_forward)   
        self.label = tk.Label(self,text = '',font="Times 24",height=1,width=14)        
        
        self.btn_draw()
        
        self.button_back.grid(row=0,column=0)
        self.button_erase.grid(row=0,column=2)
        self.button_draw.grid(row=0,column=3)
        self.button_forward.grid(row=0,column=1)
        self.button_clear.grid(row=0,column=4)
        self.canv.grid(row=1,column=0,columnspan=5)
        self.label.grid(row=2,column=0,columnspan=5)
        
        self.label_text = ''
        self.dic = {0:'0',1:'1',2:'2',3:'3',4:'4',5:'5',6:'6',7:'7',8:'8',9:'9',\
                   10:'+',11:'-',12:'*',13:'/',14:'(',15:')'}
        
        self.main_array = [[]]
        self.coord = []
        self.label_array = []
        self.position = 0
        self.color = 'green'
        self.width = 5
        self.rect_d = 3
        self.points = 2
        
    def xy(self,event):        
        self.last_x, self.last_y = self.canv.canvasx(event.x), self.canv.canvasy(event.y)        
        self.coord.append((self.last_x,self.last_y))
        
    def add_line(self,event,color,width):      
        x, y = self.canv.canvasx(event.x), self.canv.canvasy(event.y)
        self.canv.create_line((self.last_x,self.last_y,x,y), fill=color,width=width) 
        self.coord.append((x,y))      
        self.last_x, self.last_y = x, y
    
    def rect_create(self,event):
        new_rect = self.add_element()
        self.position_correct()
        self.main_array.insert(0,copy.deepcopy(self.main_array[0]))
        self.rectangles_check(self.main_array[0],new_rect)
        self.canvas_update()
        self.coord = []
        
    def add_element(self):
        t = np.array(self.coord)
        t1 = np.min(t,axis=0)
        t2 = np.max(t,axis=0)     
        rect_tuple = (t1[0]-self.rect_d, t1[1]-self.rect_d, t2[0]+self.rect_d, t2[1]+self.rect_d)
        self.canv.create_rectangle(rect_tuple)
        return [[self.coord],rect_tuple,'&']
        
    def rect_delete(self,event):
        self.canv.delete('line')
        self.spline_check(self.main_array[self.position],self.coord)
        self.coord = []
    
    def rectangles_check(self,a,b):
        def intersect_check(t1,t2):            
            if t2[1] > t1[3]:
                return 1
            elif t2[3] < t1[1]:
                return 0
            elif t2[0] < t1[0]:
                return 0
            else:
                return 1
              
        for i in self.coord:
            if len(a) > 0:
                tmp = 0
                for j in a:
                    if i[0] > j[1][0] and i[1] > j[1][1]\
                    and i[0] < j[1][2] and i[1] < j[1][3]:
                        b[0] += j[0]
                        b[1] = (min(j[1][0],b[1][0]),min(j[1][1],b[1][1]),\
                               max(j[1][2],b[1][2]),max(j[1][3],b[1][3]))
                        a.pop(tmp)
                    tmp += 1
            else:
                break
        
        for i in range(len(a)):
            y_check = intersect_check(a[i][1],b[1])
            if y_check == 0:
                a.insert(i,b)
                return a      
        a.append(b)
        return a                  
    
    def spline_check(self,a,coord):
        t = copy.deepcopy(a)
        len_t = len(t)
        z = 0
        for i in coord:
            if len_t > 0:
                for j in range(len(t)):
                    if i[0] > t[j][1][0] and i[1] > t[j][1][1]\
                    and i[0] < t[j][1][2] and i[1] < t[j][1][3]:
                        t.pop(j)
                        z = 1
                        break
            else:
                break
        
        if z == 1:
            self.position_correct()
            self.main_array.insert(0,t)
            self.canvas_update()
    
    def position_correct(self):
        for i in range(self.position):
            self.main_array.pop(0)
        self.position = 0
            
    def number_create(self,rectangle):
        def d2(delta):
            if delta[0]*ratio < delta[1]:
                return ((delta[1]/ratio-delta[0])//2,0)
            else:
                return (0,(delta[0]*ratio-delta[1])//2)
        
        re_min_x = rectangle[1][0] + self.rect_d
        re_min_y = rectangle[1][1] + self.rect_d
        re_max_x = rectangle[1][2] - self.rect_d
        re_max_y = rectangle[1][3] - self.rect_d
        delta = (re_max_x - re_min_x, re_max_y - re_min_y)
        max_delta = max(delta)
        c = d2(delta)
        x_c = lambda x: round( (x+c[0]-re_min_x) * (size_p[0]-2*self.points) / max_delta) +2
        y_c = lambda y: round( (y+c[1]-re_min_y) * (size_p[1]-2*self.points) / max_delta) +2
        square = np.zeros(size_p[0]*size_p[1])
        for j in rectangle[0]:
            for i in j:
                color = 1
                x = x_c(i[0])
                y = y_c(i[1])
                square[x + y*size_p[0]] = color
                for k in range(1,self.points):
                    square[x-k + y*size_p[0]] = color
                    square[x-k + (y-k)*size_p[0]] = color/2
                    square[x+k + y*size_p[0]] = color
                    square[x+k + (y+k)*size_p[0]] = color/2
                    square[x + (y-k)*size_p[0]] = color
                    square[x-k + (y-k)*size_p[0]] = color/2
                    square[x + (y+k)*size_p[0]] = color
                    square[x+k + (y+k)*size_p[0]] = color/2
        return np.argmax(n.query(square))
                    
        
    def canvas_update(self):
        self.label_text = ''
        self.label_array = []
        self.canv.delete('all')
        for i in self.main_array[self.position]:
            if i[2] == '&':
                i[2] = self.number_create(i)
            self.label_text += self.dic[i[2]]
            self.label_array.append(i[2])
            
            self.canv.create_rectangle(i[1])
            for k in range(len(i[0])):
                for j in range(len(i[0][k])-1):
                    self.canv.create_line((i[0][k][j][0],i[0][k][j][1],i[0][k][j+1][0],\
                                           i[0][k][j+1][1]),fill=self.color,width=self.width)
        self.label_update()
        
    def label_update(self):
        self.label.config(text = self.label_text)
        ()
        
    
    def btn_back(self): 
        if self.position < len(self.main_array)-1:
            self.position += 1
            self.canvas_update()
        
    def btn_forward(self):
        if self.position > 0:
            self.position -= 1
            self.canvas_update()
    
    def btn_erase(self):
        self.button_erase.config(background="red")
        self.button_draw.config(background="white")
        self.b1_m = self.canv.bind("<B1-Motion>", lambda event: self.add_line(event,'red',1))
        self.b1_br = self.canv.bind("<B1-ButtonRelease>", self.rect_delete)
    
    def btn_draw(self):
        self.button_erase.config(background="white")
        self.button_draw.config(background="yellow")
        self.b1 = self.canv.bind("<Button-1>", self.xy)
        self.b1_m = self.canv.bind("<B1-Motion>", \
                                   lambda event: self.add_line(event,self.color,self.width))
        self.b1_br = self.canv.bind("<B1-ButtonRelease>", self.rect_create)
                
    def btn_clear(self):
        self.position = len(self.main_array)-1
        self.canvas_update()
                               
App().mainloop()